# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv as csv

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [71]:


# Load datasets from the 'data' folder
aviation_df = pd.read_csv('data/AviationData.csv', encoding='ISO-8859-1', low_memory=False)
statecodes_df=pd.read_csv('data/USState_Codes.csv', encoding='ISO-8859-1')


# Standardize column names
aviation_df.columns = aviation_df.columns.str.lower().str.replace('.', '_')

def inspect_data(df, name):
    print(f"\n{'='*50}")
    print(f"INSPECTION: {name}")
    print(f"{'='*50}")
    
    print("\n--- Basic Info ---")
    # For info() in notebooks, capture the output as string isn't ideal
    # So I just call it directly
    df.info()
    
    print("\n--- Missing Values ---")
    print(f"Total NaNs: {df.isnull().sum().sum()}")
    nan_counts = df.isnull().sum()
    # Only show columns that have NaNs
    if nan_counts.sum() > 0:
        print(nan_counts[nan_counts > 0])
    else:
        print("No missing values found")
    
    print("\n--- Summary Statistics ---")
    print(df.describe(include='all'))
    print(f"\n{'='*50}\n")

# Then call the function on each dataframe
inspect_data(aviation_df, "Aviation Data")
inspect_data(statecodes_df, "US State Codes")


INSPECTION: Aviation Data

--- Basic Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event_id                88889 non-null  object 
 1   investigation_type      88889 non-null  object 
 2   accident_number         88889 non-null  object 
 3   event_date              88889 non-null  object 
 4   location                88837 non-null  object 
 5   country                 88663 non-null  object 
 6   latitude                34382 non-null  object 
 7   longitude               34373 non-null  object 
 8   airport_code            50132 non-null  object 
 9   airport_name            52704 non-null  object 
 10  injury_severity         87889 non-null  object 
 11  aircraft_damage         85695 non-null  object 
 12  aircraft_category       32287 non-null  object 
 13  registration_number     87507 non-null  obje

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [ ]:

def filter_aviation_data(aviation_df, start_year=1983):
    """
    Filter aviation data based on client requirements:
    - Focus on makes and models
    - Focus on professional builds (exclude homebuilt/amateur aircraft)
    - Only include data from 1983 onwards (assuming 40-year max lifetime)
    - Filter for relevant information about aircraft destruction and injuries
    
    Parameters:
    -----------
    aviation_df : pandas DataFrame
        The original aviation accident dataset
    start_year : int, default=1983
        The starting year for filtering (based on 40-year max lifetime)
        
    Returns:
    --------
    pandas DataFrame
        Filtered dataset containing relevant accident information
    """
    
    # Create a copy to avoid modifying the original dataframe
    filtered_df = aviation_df.copy()
    
    # Extract year from the event date if it exists
    if 'event_date' in filtered_df.columns:
        filtered_df['year'] = pd.to_datetime(filtered_df['event_date'], errors='coerce').dt.year

    
    # Filter for data from 1983 onwards
    filtered_df = filtered_df[filtered_df['year'] >= start_year]
    
    # Exclude amateur/homebuilt aircraft if that information exists
    filtered_df = filtered_df[filtered_df['amateur_built'] != 'Yes']
    
    # Return the filtered dataframe
    return filtered_df

# Apply the filter to the aviation data
filtered_aviation_df = filter_aviation_data(aviation_df)

#Clean and simplify some columns 
filtered_aviation_df['make'] = filtered_aviation_df['make'].str.lower().str.replace(' ', '_')
filtered_aviation_df['model'] = filtered_aviation_df['model'].str.lower().str.replace(' ', '_').str.replace('-','_')
filtered_aviation_df['injury_severity']=filtered_aviation_df['injury_severity'].str.lower().str.strip()
filtered_aviation_df['aircraft_damage'] = filtered_aviation_df['aircraft_damage'].str.lower().str.strip()


# Examine the filtered data
inspect_data(filtered_aviation_df, "Filtered Aviation Data (1983-2023, Professional Builds)")

# Now create separate datasets for small aircraft vs larger passenger models
# We'll need to define what constitutes "small" vs "large" based on available columns
# This might be based on number of engines

def categorize_aircraft_size(filtered_df):
    """
    Categorize aircraft into small vs larger passenger models
    
    Parameters:
    -----------
    filtered_df : pandas DataFrame
        The filtered aviation dataset
        
    Returns:
    --------
    tuple of pandas DataFrames
        (small_aircraft_df, large_aircraft_df)
    """
    # Create copies to avoid modifying the original dataframe
    aircraft_df_with_airplane_size = filtered_df.copy()
    
    # Depending on available columns, we'll categorize by:
    # - Number of engines (if available)
    
    aircraft_df_with_airplane_size['aircraft_size'] = aircraft_df_with_airplane_size.apply(lambda x:'small' if x['number_of_engines'] == 1 else 'large' if x['number_of_engines'] >= 2 else 'not_defined', axis = 1)
    
    return aircraft_df_with_airplane_size

# Apply the categorization
aircraft_df_with_airplane_size = categorize_aircraft_size(filtered_aviation_df)

# Examine the categorized data
inspect_data(aircraft_df_with_airplane_size, "Aircraft Data")


INSPECTION: Filtered Aviation Data (1983-2023, Professional Builds)

--- Basic Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 77061 entries, 3600 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event_id                77061 non-null  object 
 1   investigation_type      77061 non-null  object 
 2   accident_number         77061 non-null  object 
 3   event_date              77061 non-null  object 
 4   location                77010 non-null  object 
 5   country                 76851 non-null  object 
 6   latitude                30198 non-null  object 
 7   longitude               30192 non-null  object 
 8   airport_code            43385 non-null  object 
 9   airport_name            45295 non-null  object 
 10  injury_severity         76062 non-null  object 
 11  aircraft_damage         73951 non-null  object 
 12  aircraft_category       25423 non-null  object 
 13  regi

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [99]:

def clean_aircraft_safety_data(aircraft_df):
    
    """
    Analyze aircraft safety metrics by make/model
    
    Parameters:
    -----------
    aircraft_df : pandas DataFrame
        The filtered aircraft dataset
        
    Returns:
    --------
    pandas DataFrame
        Safety metrics by make/model

        injury_severity 
        total_fatal_injuries      
        total_serious_injuries    
        total_minor_injuries    
        total_uninjured            
    """
    
    # Group by make/model
    # grouped_aircraft = aircraft_df.copy()
    
    # .groupby(['make', 'model']).agg({            
    #     'total_fatal_injuries': 'sum',      # Sum of fatal injuries
    #     'total_serious_injuries': 'sum',     # Sum of serious injuries
    #     'total_minor_injuries':'sum',          # Sum of minor injuries
    #     'total_uninjured':'sum'
    # })
    
    # inspect_data(grouped_aircraft, "Make/Model Grouping")
    
    # Calculate rates of Fatal or Serious Injuries

    aircraft_df_with_injury_rate = aircraft_df.copy()
    aircraft_df_with_injury_rate['fatal_or_serious_injury_rate'] = (aircraft_df.total_fatal_injuries + aircraft_df.total_serious_injuries) / (aircraft_df.total_fatal_injuries + aircraft_df.total_serious_injuries + aircraft_df.total_minor_injuries + aircraft_df.total_uninjured)

    
    display (aircraft_df_with_injury_rate)
    return aircraft_df_with_injury_rate



    
# Analyze safety for small and large aircraft
aircraft_data_with_safety = clean_aircraft_safety_data(aircraft_df_with_airplane_size)





,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,...,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date,year,aircraft_size,fatal_or_serious_injury_rate
3600,20001214X42040,Accident,LAX83LA093,1983-01-01,"ARROYO GRANDE, CA",United States,NaN,NaN,NaN,NaN,...,1.0,0.0,1.0,VMC,Landing,Probable Cause,NaN,1983,not_defined,0.5
3601,20001214X42095,Accident,SEA83LA036,1983-01-01,"NEWPORT, OR",United States,NaN,NaN,ONP,NEWPORT MUNICIPAL,...,0.0,1.0,3.0,VMC,Approach,Probable Cause,NaN,1983,small,0.0
3602,20001214X42067,Accident,MKC83LA056,1983-01-01,"WOODBINE, IA",United States,NaN,NaN,3YR,MUNICIPAL,...,0.0,0.0,2.0,VMC,Landing,Probable Cause,NaN,1983,small,0.0
3603,20001214X42063,Accident,MKC83LA050,1983-01-01,"MARYVILLE, MO",United States,NaN,NaN,78Y,RANKIN,...,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,NaN,1983,small,0.0
3604,20001214X42018,Accident,LAX83FUG11,1983-01-01,"UPLAND, CA",United States,NaN,NaN,CCB,CABLE,...,0.0,2.0,0.0,VMC,Approach,Probable Cause,NaN,1983,small,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022,2022,not_defined,1.0
88885,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2022,not_defined,NaN
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,...,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022,2022,small,0.0
88887,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2022,not_defined,NaN


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [ ]:
def clean_aircraft_damage (aircraft_df):
    clean_aircraft_damage_df=aircraft_df.copy()

    clean_aircraft_damage_df['destroyed'] = clean_aircraft_damage_df.apply(lambda x: True if x.aircraft_damage == 'destroyed' else False if x.aircraft_damage == 'substantial' or x.aircraft_damage == 'minor' else 'unknown', axis = 1)

    return clean_aircraft_damage_df
    
aircraft_data_with_destruction = clean_aircraft_damage(aircraft_data_with_safety)

inspect_data(aircraft_data_with_destruction, 'Aircraft Data With Destruction')


INSPECTION: Aircraft Data With Destruction

--- Basic Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 77061 entries, 3600 to 88888
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   event_id                      77061 non-null  object 
 1   investigation_type            77061 non-null  object 
 2   accident_number               77061 non-null  object 
 3   event_date                    77061 non-null  object 
 4   location                      77010 non-null  object 
 5   country                       76851 non-null  object 
 6   latitude                      30198 non-null  object 
 7   longitude                     30192 non-null  object 
 8   airport_code                  43385 non-null  object 
 9   airport_name                  45295 non-null  object 
 10  injury_severity               76062 non-null  object 
 11  aircraft_damage               73951 non-null  object 
 12

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [ ]:

#create a data frame for make

def clean_make (aircraft_df):
    make_cleaned_df=aircraft_df.copy()
    make_list = make_cleaned_df['make'].tolist()

    print (make_list)
    inspect_data(make_list, 'thing')

clean_make (aircraft_data_with_destruction)


#cleaning grouped_aircraft by removing characters, using lowercase






['piccard', 'cessna', 'cessna', 'cessna', 'piper', 'cessna', 'balloon_works', 'cessna', 'north_american', 'piper', 'piper', 'beech', 'beech', 'cessna', 'cessna', 'cessna', 'swearingen', 'cessna', 'cessna', 'piper', 'canadair', 'piper', 'cessna', 'piper', 'cessna', 'douglas', 'cessna', 'north_american', 'piper', 'piper', 'cessna', 'mooney', 'beech', 'cessna', 'cessna', 'cessna', 'teal', 'bell', 'piper', 'schweizer', 'bell', 'cessna', 'bell', 'boeing', 'piper', 'cessna', 'hiller', 'cessna', 'piper', 'beech', 'cessna', 'cessna', 'bellanca', 'cessna', 'cessna', 'cessna', 'robinson', 'balloon_works', 'grumman', 'luscombe', 'piper', 'mooney', 'beech', 'piper', 'mooney', 'convair', 'cessna', 'enstrom', 'cessna', 'piper', 'bell', 'mcdonnell_douglas', 'bell', 'cessna', 'lockheed', 'champion', 'piper', 'beech', 'piper', 'bellanca', 'mcdonnell_douglas', 'piper', 'cessna', 'bell', 'piper', 'cessna', 'piper', 'cessna', 'robinson', 'cessna', 'piper', 'piper', 'grumman_american', 'cessna', 'piper', '

TypeError: inspect_data() missing 1 required positional argument: 'name'

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [ ]:
if 'Model' in df:
        # Remove common suffixes and standardize
        cleaned_df['Model'] = cleaned_df['Model'].str.replace(r'-\d+[A-Z]?$', '', regex=True)  # Remove -100, -200A etc.
        cleaned_df['Model'] = cleaned_df['Model'].str.replace(r'\s+', ' ', regex=True)  # Standardize spaces

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized